In [53]:
import glob
import csv

def parse_lyrics():
    unique_line_ratios = []
    artists = []
    titles = []
    genres = []
    lyric_strings = []
    
    #iterate through files
    for filename in glob.iglob('data/*.txt'):
        print("### Parsing file " + filename)
        genre = filename[5:len(filename)-4]
        numberOfSongs = 0
        file = open(filename)
        lines = file.readlines()
        i = 0
        while i < len(lines) and not lines[i].startswith("###"):
            i = i + 1
        while i < len(lines):
            numberOfSongs = numberOfSongs + 1
            artists.append(lines[i][3:].strip())
            titles.append(lines[i+1][3:].strip())
            i = i + 2
            #extract features from song
            
            lyrics = []
            while i < len(lines) and not lines[i].startswith("###"):
                line = lines[i].strip()
                if line != "":
                    lyrics.append(line)
                i = i + 1
                
            lyric_strings.append(make_lyric_string(lyrics))   
            unique_line_ratios.append(calc_unique_line_ratio(lyrics))
            genres.append(genre)    
    
    return unique_line_ratios, artists, titles, genres, lyric_strings;

def calc_unique_line_ratio(lyrics):
    return len(set(lyrics)) / len(lyrics)

def make_lyric_string(lyrics):
    lyric_string = ''
    for l in lyrics:
        lyric_string += l+' '
    return lyric_string 

In [61]:
from sklearn.feature_extraction.text import CountVectorizer

unique_line_ratios, artists, titles, genres, lyrics = parse_lyrics()
vectorizer = CountVectorizer()
bag_of_words = vectorizer.fit_transform(lyrics)

### Parsing file data\Blues.txt
### Parsed 100 songs for Blues

### Parsing file data\Country.txt
### Parsed 100 songs for Country

### Parsing file data\EDM.txt
### Parsed 100 songs for EDM

### Parsing file data\Metal.txt
### Parsed 100 songs for Metal

### Parsing file data\Rap.txt
### Parsed 40 songs for Rap

### Parsing file data\Rock.txt
### Parsed 0 songs for Rock

